# 2. Arrays and linear algebra

## Arrays

Arrays are the standard "container" objects, representing lists/vectors, matrices and higher order tensors.

Arrays can be constructed using the literal syntax:

In [1]:
X = [1,2,3] # a vector

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
M = [1 2 3; 4 5 6; 7 8 9] # matrix: note the different separators

3x3 Array{Int64,2}:
 1  2  3
 4  5  6
 7  8  9

Arrays are a parametric type with 2 parameters: 
* the first is the type of the element: this can be either concrete or abstract.
* the second is the number of dimensions.

The literal syntax will attempt to find a "common" type

In [3]:
[1, 2.0, 3]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

In [4]:
[1, 2.0, 3+4im]

3-element Array{Complex{Float64},1}:
 1.0+0.0im
 2.0+0.0im
 3.0+4.0im

You can also prefix with the desired type:

In [5]:
Any[1,2,3]

3-element Array{Any,1}:
 1
 2
 3

In [6]:
Float64[] # useful for constructing empty arrays

0-element Array{Float64,1}

`Vector` and `Matrix` are aliases for 1- and 2-dimensional `Array`s:

In [7]:
Vector{Float64}

Array{Float64,1}

In [8]:
Matrix{Float64}

Array{Float64,2}

> **Performance note:** there are performance and memory advantages to using Arrays with a concrete type (e.g. `Array{Float64,1}` vs `Array{FloatingPoint,1}` or `Array{Any,1}`), as the compiler is able to infer the type without looking at each element, and able to store them more efficiently.

### Indexing
Indexing is 1-based (like R and Matlab, not 0-based like Python or C), and via square brackets:

In [9]:
X = [5,6,7]
X[2]

6

In [10]:
X[3] = 8 # replace 3rd element
X

3-element Array{Int64,1}:
 5
 6
 8

Matrices are stored in column-major order, and can be indexed by either 2 indices (row, column) or 1 index (raw ordering of the storage)

In [11]:
M = [1 2 3; 4 5 6; 7 8 9]
M[2,1] # 2nd row, 1st column

4

In [12]:
M[3] # 3rd element in column-major storage

7

### Array operations

Julia distinguishes between functions that are applied to an Array, and functions that are applied to elements of an array.

Operators preceded by a dot (`.`) are applied elementwise:

In [13]:
X = [1,2,3]; Y = [1,4,2]

X == Y # true if and only if all elements are equal

false

In [14]:
X .== Y # compares each element

3-element BitArray{1}:
  true
 false
 false

Same for arithmetic operators:

In [15]:
X .* Y # element-wise multiplication

3-element Array{Int64,1}:
 1
 8
 6

In [16]:
M * Y # matrix multiplication

3-element Array{Int64,1}:
 15
 36
 57

One notable difference to other languages is that `min`/`max` are applied elementwise, and `minimum`/`maximum` are reducing (similar to `+` and `sum`)

In [17]:
min(X,Y)

3-element Array{Int64,1}:
 1
 2
 2

In [18]:
minimum(X)

1

### Other methods for constructing arrays

* `zeros`/`ones` constructs arrays of zeros/ones
* `rand`/`randn` constructs arrays of random U(0,1) or N(0,1) variates
* `eye` constructs identity matrices

In [19]:
zeros(4) # note difference from matlab

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [20]:
rand(3,4)

3x4 Array{Float64,2}:
 0.47363    0.447494  0.844031  0.617446
 0.0033705  0.869553  0.780045  0.772283
 0.759368   0.994455  0.665182  0.625277

In [21]:
ones(Int,3) # optional first argument specifies type

3-element Array{Int64,1}:
 1
 1
 1

In [22]:
eye(4) # gives matrix, not vector

4x4 Array{Float64,2}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

It is also possible to call `Array` directly, though you may get unexpected results:

In [23]:
Array(Float64,3,3)

3x3 Array{Float64,2}:
 6.44724e-314  6.44721e-314  0.0
 6.44727e-314  6.44727e-314  0.0
 6.44727e-314  6.44723e-314  0.0

> **Question:**
> 1. Why does this happen?

## Abstract Arrays and Ranges

`Array` is a subtype of `AbstractArray`, which captures all "array-like" objects. The simplest example are the `Range` objects:

In [24]:
R = 0:9 # ranges are specied by start:end

0:9

In [25]:
R = 0:0.1:1 # optional middle element specifies step

0.0:0.1:1.0

In [26]:
R[4] # like arrays we can index

0.3

In [27]:
R[4] = 7 # can't reassign values

LoadError: LoadError: indexed assignment not defined for FloatRange{Float64}
while loading In[27], in expression starting on line 1

In [28]:
2+3*R # can use usual arithmetic operations

2.0:0.3:5.0

Some other `AbstractArray`s:
* `BitArray`: efficient storage of an array of `Bool`s (`true`/`false`)
* `SparseMatrixCSC`: sparse matrix stored in "Compressed Sparse Column" format
* `Diagonal`/`Bidiagonal`/`Tridiagonal`: efficient storage of special matrix types
* `UpperTriangular`/`LowerTriangular`: triangular "mask" over a matrix

In [29]:
UpperTriangular(ones(3,3))

3x3 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  1.0  1.0
 0.0  1.0  1.0
 0.0  0.0  1.0

### Comprehensions

Comprehensions provide convenient and efficient ways to construct arrays from iterable objects.

In [30]:
[exp(x^2/2) for x = -1:0.5:1]

5-element Array{Float64,1}:
 1.64872
 1.13315
 1.0    
 1.13315
 1.64872

> **Performance tip**: this is actually much more efficient than the equivalent vectorised operation (`exp((-1:0.5:1).^2./2)`) as it doesn't require allocating any intermediate arrays.

Comprehensions over multiple objects give arrays:

In [31]:
[i+j for i=1:5, j=1:5]

5x5 Array{Int64,2}:
 2  3  4  5   6
 3  4  5  6   7
 4  5  6  7   8
 5  6  7  8   9
 6  7  8  9  10

In [32]:
[c for c in "αβγ"] # can be used with any iterable object of known length

3-element Array{Char,1}:
 'α'
 'β'
 'γ'

## Linear Algebra

Basic arithmetic operations:
* `*` for matrix multiplication
* `\` and `/` for matrix division
* `'` is for conjugate transpose (`.'` for non-conjugate version)

In [33]:
M = randn(4,4); x = randn(4)
M' \ x # equivalent to inv(M') * x

4-element Array{Float64,1}:
  0.207792
 -0.769971
  0.370108
 -1.231   

Standard factorizations are available:
* `chol`, `qr`, `svd`, `lu`

In [34]:
X = randn(7,4)
Q,R = qr(X)

(
7x4 Array{Float64,2}:
 -0.0827846   0.757994    0.205199  -0.445744 
  0.102877   -0.247672   -0.232127   0.0142528
  0.351658    0.119107   -0.845958  -0.217754 
 -0.433929   -0.335476   -0.10832   -0.6105   
  0.691437    0.0823038   0.216027   0.0486207
 -0.118959    0.406847   -0.240689   0.0038417
 -0.422336    0.255099   -0.268392   0.615308 ,

4x4 Array{Float64,2}:
 -1.62945  -1.03232  -0.437897  -0.103966
  0.0      -1.11043   0.392996  -1.37593 
  0.0       0.0      -2.01903   -0.974127
  0.0       0.0       0.0       -0.683027)

However you often don't need these.

Julia also has `Factorisation` objects: these perform the factorisation, but keep the object stored in a more efficient format (e.g. the Q matrix is stored as Householder reflectors).

In [35]:
F = qrfact(X)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}}(7x4 Array{Float64,2}:
 -1.62945    -1.03232    -0.437897   -0.103966 
 -0.0950114  -1.11043     0.392996   -1.37593  
 -0.324772   -0.310705   -2.01903    -0.974127 
  0.400753    0.543735    0.0406088  -0.683027 
 -0.638573   -0.481722   -0.132565    0.213176 
  0.109864   -0.275226    0.174357   -0.0783506
  0.390047    0.0344947   0.184837   -0.612737 ,4x4 Array{Float64,2}:
 1.08278       -0.655117  -0.0570885   0.370343
 0.0            1.17565    0.577913   -0.634615
 0.0            0.0        1.84538     0.29631 
 2.22788e-314   0.0        0.0         1.40151 )

In [36]:
F.factors

7x4 Array{Float64,2}:
 -1.62945    -1.03232    -0.437897   -0.103966 
 -0.0950114  -1.11043     0.392996   -1.37593  
 -0.324772   -0.310705   -2.01903    -0.974127 
  0.400753    0.543735    0.0406088  -0.683027 
 -0.638573   -0.481722   -0.132565    0.213176 
  0.109864   -0.275226    0.174357   -0.0783506
  0.390047    0.0344947   0.184837   -0.612737 

These are particularly useful when
* you can exploit the structure of the array: e.g. you know the matrix is symmetric positive definite, so can use `cholfact(M)\x`
* you reuse the same matrix, e.g. perform `qrfact` outside the loop, then reuse the object inside.

By convention, methods ending in a ! mutate their arguments: this is particularly useful for large arrays as it can save memory:

In [37]:
Y=randn(6,2)

6x2 Array{Float64,2}:
 -0.201024   0.34082 
 -1.40989    0.455595
  2.02248   -1.4306  
 -0.967738   1.32365 
 -0.376252   0.409193
  1.26405    0.914169

In [38]:
F = qrfact!(Y)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}}(6x2 Array{Float64,2}:
  2.96555   -1.30956 
  0.445241   1.84678 
 -0.638695   0.1771  
  0.30561   -0.38536 
  0.11882   -0.100233
 -0.399184  -0.739875,2x2 Array{Float64,2}:
 1.06779       -0.611903
 2.19682e-314   1.15119 )

In [39]:
Y

6x2 Array{Float64,2}:
  2.96555   -1.30956 
  0.445241   1.84678 
 -0.638695   0.1771  
  0.30561   -0.38536 
  0.11882   -0.100233
 -0.399184  -0.739875

Julia's linear algebra routines also work for other numeric types:

In [40]:
X = [1//3 3//5 6//2; 1//2 3//2 8//7; 2//1 9//2 3//8]

3x3 Array{Rational{Int64},2}:
 1//3  3//5  3//1
 1//2  3//2  8//7
 2//1  9//2  3//8

In [41]:
F = lufact(X)

Base.LinAlg.LU{Rational{Int64},Array{Rational{Int64},2}}(3x3 Array{Rational{Int64},2}:
 1//3  3//5     3//1 
 3//2  3//5   -47//14
 6//1  3//2  -705//56,[1,2,3],0)

In [42]:
F \ [1,2,3]

3-element Array{Rational{Int64},1}:
 -393//94
    5//2 
   14//47